In [1]:
import pandas
import gripql
import itertools
import scipy.stats as stats

conn = gripql.Connection("https://bmeg.io/api", credential_file="bmeg_credentials.json")
O = conn.graph("bmeg_rc2")

Find all of the samples in the [CTRP](https://portals.broadinstitute.org/ctrp/) Breast Cancer experiment

In [2]:
q = O.query().V("Project:CTRP_Breast_Cancer").out("cases").distinct("_gid")
all_cases = []
for row in q:
    all_cases.append(row.gid)

[INFO]	2019-07-26 20:29:14,278	40 results received in 0 seconds


For the genes of interest, get Ensembl gene ids, from the HUGO symbols

In [3]:
GENES = ["PTEN", "TP53"]

In [4]:
gene_ids = {}
for i in O.query().V().hasLabel("Gene").has(gripql.within("symbol", GENES)):
    gene_ids[i.data.symbol] = i.gid

[INFO]	2019-07-26 20:29:14,423	2 results received in 0 seconds


The CTRP doesn't have direct mutation calling, but rather they used the same cell lines as the CCLE, and we can use the mutation calls from that project. So use the `same_as` edge to identify the cases from CCLE that are the same as the ones in CTRP. Then use the mutations from those samples.

In [5]:
gene_ids

{'PTEN': 'ENSG00000171862', 'TP53': 'ENSG00000141510'}

For each of the genes, find the set of samples that have a mutation in that gene

In [6]:
mut_cases = {}
norm_cases = {}

q = O.query().V(all_cases).as_("ctrp").out("same_as").has(gripql.eq("project_id", "Project:CCLE_Breast_Cancer"))
q = q.out("samples").out("aliquots").out("somatic_callsets")
q = q.outE("alleles").has(gripql.within("ensembl_gene", list(gene_ids.values())))
q = q.render({"case" : "$ctrp._gid", "gene" : "$._data.ensembl_gene"})

for res in q:
    mut_cases[res.gene] = mut_cases.get(res.gene, set()) | set([res.case])

#get CCLE samples without mutation
for i in gene_ids.values():
    norm_cases[i] = list(set(all_cases).difference(mut_cases[i]))

    print( "%s Positive Set: %d" % (i, len(mut_cases[i])) )
    print( "%s Negative Set: %d" % (i, len(norm_cases[i])) )


[INFO]	2019-07-26 20:29:14,690	43 results received in 0 seconds


ENSG00000171862 Positive Set: 9
ENSG00000171862 Negative Set: 31
ENSG00000141510 Positive Set: 29
ENSG00000141510 Negative Set: 11


In [7]:
pos_response = {}
for g in gene_ids.values():
    pos_response[g] = {}
    q = O.query().V(list(mut_cases[g])).as_("a").out("samples").out("aliquots")
    q = q.out("drug_response").as_("a").out("compounds").as_("b")
    q = q.select(["a", "b"])    
    for row in q:
        v = row['a']['data']['auc']
        compound = row['b']['gid']
        if compound not in pos_response[g]:
            pos_response[g][compound] = [ v ]
        else:
            pos_response[g][compound].append(v)
   

[INFO]	2019-07-26 20:29:16,035	3,592 results received in 1 seconds
[INFO]	2019-07-26 20:29:19,540	12,224 results received in 3 seconds


In [8]:
neg_response = {}
for g in gene_ids.values():
    neg_response[g] = {}
    q = O.query().V(list(norm_cases[g])).as_("a").out("samples").out("aliquots")
    q = q.out("drug_response").as_("a").out("compounds").as_("b")
    q = q.select(["a", "b"])    
    for row in q:
        v = row['a']['data']['auc']
        compound = row['b']['gid']
        if compound not in neg_response[g]:
            neg_response[g][compound] = [ v ]
        else:
            neg_response[g][compound].append(v)
   

[INFO]	2019-07-26 20:29:23,489	13,250 results received in 3 seconds
[INFO]	2019-07-26 20:29:25,137	4,618 results received in 1 seconds


In [9]:
drugs = set(itertools.chain.from_iterable( i.keys() for i in pos_response.values() ))
out = []
for drug in drugs:
    for g in gene_ids.values():
        if drug in pos_response[g] and drug in neg_response[g]:
            row = {"drug" : drug, "mutation" : g}
            mut_values = pos_response[g][drug]
            norm_values = neg_response[g][drug]
            if len(mut_values) > 5 and len(norm_values) > 5:
                s = stats.ttest_ind(mut_values, norm_values, equal_var=False)
                row["t-statistic"] = s.statistic
                row["t-pvalue"] = s.pvalue
                s = stats.f_oneway(mut_values, norm_values)
                row["a-statistic"] = s.statistic
                row["a-pvalue"] = s.pvalue
                out.append(row)

In [11]:
pandas.DataFrame(out, columns=["drug", "mutation", "t-statistic", "t-pvalue", "a-statistic", "a-pvalue"]).sort_values("a-pvalue").head(10)

,drug,mutation,t-statistic,t-pvalue,a-statistic,a-pvalue
369,Compound:CID9967941,ENSG00000141510,4.098840,0.000438,12.705330,0.001136
136,Compound:CID11433190,ENSG00000141510,2.941631,0.011004,12.336689,0.001189
315,Compound:CID56949517,ENSG00000171862,-2.611946,0.041333,13.650130,0.001197
191,Compound:CID11152667,ENSG00000141510,-3.003887,0.008382,9.205017,0.004602
41,Compound:NO_ONTOLOGY~BRD-K97651142,ENSG00000141510,2.556805,0.023745,8.381266,0.006489
44,Compound:CID11609586,ENSG00000171862,-1.952776,0.068235,7.382291,0.008403
649,Compound:CID52947560,ENSG00000141510,2.319721,0.036225,7.511026,0.009386
682,Compound:CID46943432,ENSG00000171862,-2.160075,0.055895,7.443883,0.009680
538,Compound:CID5566,ENSG00000141510,1.736441,0.118978,7.362028,0.010777
377,Compound:CID31703,ENSG00000171862,-2.835380,0.009463,6.628285,0.012186
